## Reading generated images

In [ ]:
import io
import cv2
import json
import time
import requests
import numpy as np
from matplotlib import pyplot as plt
from random import sample as samp
from IPython.display import clear_output
from lib.util import to_ai_url, setup_plt
from lib.load import index_urls, gen_images
from lib.util import setup_viz_box, draw_viz_box
from lib.truth import to_truth
from lib.ai import gen_results
from lib.stats import to_ious

USE_AWS_AI = True

print("testing", "on aws" if (USE_AWS_AI) else "locally")
plt.rcParams["figure.figsize"] = (16,16)
plt.rcParams["figure.dpi"] = 40

SPICES = [
    'allspice', 'anise', 'caraway', 'cardamom', 'celery_seed',
    'chiles', 'cinnamon', 'cloves', 'coriander', 'cumin',
    'dill_seed', 'fennel', 'fenugreek', 'ground_ginger',
    'ground_pepper', 'mace_spice', 'mustard_seed', 'nutmeg',
    'peppercorns', 'saffron', 'salt', 'sumac', 'tumeric',
    'vanilla_extract'
]

In [ ]:
NUM_READ = -1 # Unlimited
#NUM_READ = 5 # TODO

images = []
sources = index_urls('merged.json', NUM_READ)
NUM_ALL = len(sources)
if (NUM_READ < 0):
    print(f'Loading {NUM_ALL} images...')
else:
    print(f'Loading {min(NUM_READ,NUM_ALL)} images...')
image_gen = gen_images(sources)

for src in sources:
    img = None
    try: img = next(image_gen)
    except StopIteration: break
    images.append(img)
    if len(images) % 10 == 0:
        clear_output(wait=True)
        print(round(100*len(images)/len(sources)),'%')

print('Loaded', len(images), 'images!')

## OCR

In [ ]:
try: images
except NameError:
    raise(Exception("Please run prior cell"))
try: USE_AWS_AI
except NameError:
    raise(Exception("Please run first cell"))

results = []
for res in gen_results(USE_AWS_AI, images):
    results.append(res)
    clear_output(wait=True)
    print(round(100*len(results)/len(images)),'%')

## Visualization

In [ ]:
try: sources and images and results
except NameError:
    raise(Exception("Please run prior cells"))

N_VIZ = 5
DELAY_VIZ = 1.0

truth = to_truth(sources)
samples = list(zip(truth, images, results))
print(f'sample size: {len(samples)}')

for (i, (real, img, res)) in enumerate(samples):
    res_text = res["ocr_all_results"][0]["text"]
    res_box = res["ocr_all_results"][0]["box"]
    rea_text = real["text"]
    rea_box = real["box"]
    if (i >= N_VIZ): continue
    time.sleep(DELAY_VIZ)
    setup_plt()
    clear_output(wait=True)
    plt.imshow(img)

    # Title to viz box plot
    viz_setup = setup_viz_box(res_text, rea_text)

    # Draw predicted bounding boxes
    draw_viz_box('predicted', [res_box], viz_setup["predicted"])

    draw_viz_box('actual', rea_box, viz_setup["actual"])
        
    plt.legend(fontsize = 20)
    plt.show()
print('DONE')


# Evaluation

In [ ]:
try: images and truth and results
except NameError:
    raise(Exception("Please run prior cells"))
    
ious = to_ious(images, truth, results)

setup_plt(True)
bin_range = list(range(0,100,5))
plt.gca().xaxis.grid(True, color='k')
plt.gca().yaxis.grid(True)
plt.hist(ious, bins = bin_range, density = False)
plt.title(f'Histogram of IOU (intersection over union)', fontsize = 50)
plt.tick_params(axis='both', which='major', labelsize = 30)
plt.xticks(bin_range, [f'{x}' for x in bin_range])

plt.xticks(bin_range)
plt.xlabel("Intersection as % of union", fontsize=40)
plt.ylabel("Occurrences", fontsize=40)
plt.show() # TODO
    

In [ ]:
try: ious
except NameError:
    raise(Exception("Please run prior cell"))
    
MEAN_IOU = np.mean(ious)

TEXT_EQUALITY = [a["text"] == b['ocr_all_results'][0]["text"] for (a, b) in zip(truth, results)]
TEXT_ACCURACY = int(np.array(TEXT_EQUALITY, dtype=bool).sum())/len(TEXT_EQUALITY)
MEAN_CORRECT_IOU = np.mean(np.array(ious)[TEXT_EQUALITY])

print(f'{round(MEAN_IOU, 3)}% mean IOU')
print(f'   {round(MEAN_CORRECT_IOU, 3)}% for true predictions')
print(f'\n{round(TEXT_ACCURACY*100, 3)}% text accuracy')

In [ ]:
try: TEXT_EQUALITY and results
except NameError:
    raise(Exception("Please run prior cell"))

N_GPT = 5
DELAY_GPT = 5.0

ai_server_url = to_ai_url(USE_AWS_AI)
print(f'Using Custom AI: {ai_server_url}')

#api request instructions
def chat_with_gpt(spices):
    endpoint = f'{ai_server_url}recipe'
    response = requests.post(endpoint, json.dumps({
        "spices": spices
    }))
    res_data = response.json()
    return res_data
  
found_spices = set([v['ocr_all_results'][0]["text"] for (t,v) in zip(TEXT_EQUALITY, results) if t])
spice_lists = [samp(list(found_spices),samp([2,3,4],1)[0]) for _ in range(N_GPT)]
    
for spices in spice_lists:
    clear_output(wait=True)
    print('INPUT SPICES:')
    print('-', ', '.join(spices))
    output = chat_with_gpt(spices)
    print('\nOUTPUT RECIPE:')
    print(f'{len(output["spices"]) - len(spices)} added spices')
    print(f'{len(output["ingredients"])} added ingredients')
    print()
    print(output["text"])
    time.sleep(DELAY_GPT)
    print('\n')